In [2]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('3_df_pak.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   job_title_short        1371 non-null   object 
 1   job_title              1371 non-null   object 
 2   job_location           1371 non-null   object 
 3   job_via                1371 non-null   object 
 4   job_schedule_type      1370 non-null   object 
 5   job_work_from_home     1371 non-null   bool   
 6   search_location        1371 non-null   object 
 7   job_posted_date        1371 non-null   object 
 8   job_no_degree_mention  1371 non-null   bool   
 9   job_health_insurance   1371 non-null   bool   
 10  job_country            1371 non-null   object 
 11  salary_rate            12 non-null     object 
 12  salary_year_avg        10 non-null     float64
 13  salary_hour_avg        1 non-null      float64
 14  company_name           1371 non-null   object 
 15  job_

In [4]:
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [5]:
from eda_functions import convert_job_skills
df['job_skills'] = df['job_skills'].apply(convert_job_skills)

unique_types = df['job_skills'].apply(type).unique()
unique_types

array([<class 'list'>, <class 'float'>], dtype=object)

In [9]:
df.job_location.unique()


array(['Karachi, Pakistan', 'Pakistan', 'Islamabad, Pakistan',
       'Lahore, Pakistan', 'Anywhere', 'Hyderabad, Pakistan',
       'Jatoi, Pakistan', 'Quetta, Pakistan',
       'Islamabad Capital Territory, Pakistan', 'Multan, Pakistan',
       'Faisalabad, Pakistan', 'Rawalpindi, Pakistan',
       'Pakistan (+2 others)', 'Pakistan (+1 other)', 'Khattan, Pakistan',
       'Khushāb, Pakistan', 'Kot Radha Kishan, Pakistan',
       'Mansehra, Pakistan', 'Qila Saifullah, Pakistan',
       'Loralai, Pakistan (+128 others)', 'Shahdara, Pakistan',
       'Peshawar, Pakistan', 'Kharian, Pakistan', 'Layyah, Pakistan',
       'Khyber Pakhtunkhwa, Pakistan', 'Sahiwal, Pakistan'], dtype=object)

In [12]:
# Function to extract city name
def extract_city(location):
    if ',' in location:
        return location.split(',')[0]
    elif ' (+' in location:
        return location.split(' (+')[0]
    elif 'Anywhere' in location:
        return 'Anywhere'
    else:
        return location

# Create new column 'city'
df['city'] = df['job_location'].apply(extract_city)

df.city.unique()

array(['Karachi', 'Pakistan', 'Islamabad', 'Lahore', 'Anywhere',
       'Hyderabad', 'Jatoi', 'Quetta', 'Islamabad Capital Territory',
       'Multan', 'Faisalabad', 'Rawalpindi', 'Khattan', 'Khushāb',
       'Kot Radha Kishan', 'Mansehra', 'Qila Saifullah', 'Loralai',
       'Shahdara', 'Peshawar', 'Kharian', 'Layyah', 'Khyber Pakhtunkhwa',
       'Sahiwal'], dtype=object)

In [15]:
df.query('city == "Anywhere"').job_work_from_home.unique()

array([ True])

It is confiremd that job posting having anywhere are work-from-home jobs

In [18]:
df.query('city == "Pakistan"')

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills,city
1,Data Engineer,Senior Consultant - Data Engineer,Pakistan,via Jooble,Full-time,False,Pakistan,2023-09-13 13:14:17,True,False,Pakistan,NaN,NaN,NaN,"ThoughtWorks, Inc.","[nosql, cassandra, aws, azure, kafka, spark, h...","{'cloud': ['aws', 'azure'], 'databases': ['cas...",Pakistan
14,Data Engineer,Remote Associate Data Engineer Jobs,Pakistan,via Jobiflix.online,Full-time,False,Pakistan,2023-01-01 13:40:46,False,False,Pakistan,NaN,NaN,NaN,Turing.com,"[java, python, sql, aws, spring, slack]","{'cloud': ['aws'], 'libraries': ['spring'], 'p...",Pakistan
27,Data Scientist,Data Scientist,Pakistan,via LinkedIn,Full-time,False,Pakistan,2023-03-03 13:11:52,False,False,Pakistan,NaN,NaN,NaN,HR Ways - Hiring Tech Talent,"[php, python, sql, r, azure, keras, pytorch, p...","{'cloud': ['azure'], 'libraries': ['keras', 'p...",Pakistan
33,Business Analyst,Analyst,Pakistan,via LinkedIn,Full-time,False,Pakistan,2023-12-15 13:15:00,False,False,Pakistan,NaN,NaN,NaN,Medtronic,[excel],{'analyst_tools': ['excel']},Pakistan
34,Business Analyst,Analyst,Pakistan,via Jooble,Full-time,False,Pakistan,2023-08-01 13:30:41,False,False,Pakistan,NaN,NaN,NaN,Sandoz AG,"[sas, sas, r, python, alteryx, sharepoint, tab...","{'analyst_tools': ['sas', 'alteryx', 'sharepoi...",Pakistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Machine Learning Engineer,Lead Machine Learning Engineer,Pakistan,via Ai-Jobs.net,Full-time,False,Pakistan,2023-05-12 20:52:41,False,False,Pakistan,year,79200.0,NaN,NorthBay Solutions,"[python, aws, jupyter, numpy, pandas, pyspark,...","{'cloud': ['aws'], 'libraries': ['jupyter', 'n...",Pakistan
1351,Machine Learning Engineer,"Senior Software Engineer, Machine Learning",Pakistan,via BeBee,Full-time,False,Pakistan,2023-07-24 01:24:45,False,False,Pakistan,NaN,NaN,NaN,"Motive Technologies, Inc","[python, c++, aws, terraform, docker, kubernetes]","{'cloud': ['aws'], 'other': ['terraform', 'doc...",Pakistan
1355,Business Analyst,Senior Business Analyst/Group Data Technology,Pakistan,via BeBee Pakistan,Full-time,False,Pakistan,2023-10-06 05:15:57,False,False,Pakistan,NaN,NaN,NaN,HSBC,[jira],{'async': ['jira']},Pakistan
1356,Data Analyst,Data Analyst (Fresh),Pakistan,via LinkedIn,Full-time,False,Pakistan,2023-10-12 05:17:29,False,False,Pakistan,NaN,NaN,NaN,HR Ways - Hiring Tech Talent,[php],{'programming': ['php']},Pakistan
